## Sources: Acquiring Streaming Data

Call to spark.readStream creates a `DataStreamReader` instance. This instance is in charge of managing the different stream formats and configurations provided through the builder method calls. 

If `DataStreamReader` was called with *.format(...)* method, only after calling *.load(...)* method on `DataStreamReader` instance, the options provided to the builder are validated and, if everything checks out, streaming DataFrame is returned.

If `DataStreamReader` was called with diffrent "loading" methods, such as *.json(...)*, *.csv(...)* and more, streaming DataFrame is returned immediately


### Available streaming source

1. **File-based source** <br>
    Monitors a path in a filesystem and consumes files atomically placed in it. The found files will then be parsed by the specified formatter and processed in order of file modification time.
    Supported formats are: 
        * text
        * csv
        * json
        * orc



2. **Socket source**  <br>
Establishes a client connection to a TCP server amd reads UTF-8 text data through a socket connection.
3. **Kafka source** <br>
Creates Kafka consumer able to retrieve data from Kafka.
4. **Rate source & Rate per Micro-batch source** <br>
Generates a stream of specified number of rows per second / specified number of rows per micro-batch . Each output row contains a *timestamp* and *value*.  It’s mainly intended as a testing source.
5. **Table source** <br>
Creates a Streaming DataFrame on a table


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName("SourcesSinks") \
        .config("spark.sql.warehouse.dir", "./warehouse") \
        .enableHiveSupport() \
        .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/04/20 19:08:33 WARN Utils: Your hostname, DELEQ0283302041 resolves to a loopback address: 127.0.1.1; using 172.31.227.62 instead (on interface eth0)
24/04/20 19:08:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/20 19:08:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/20 19:08:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/04/20 19:08:35 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DecimalType
# define schema
real_estate_schema = StructType(
    [StructField('UID', IntegerType()), 
    StructField('Location', StringType(), True), 
    StructField('Price', DecimalType(15,2), True), 
    StructField('Bedrooms', IntegerType(), True), 
    StructField('Bathrooms', IntegerType(), True), 
    StructField('Size', IntegerType(), True), 
    StructField('Price SQ Ft', DecimalType(10,2), True), 
    StructField('Status', StringType(), True)])

# Cleanup workspace from previous runs
spark.sql("DROP TABLE IF EXISTS real_estate")

# create table from DataFrame
real_estate_df = (
    spark
    .read
    .schema(real_estate_schema)
    .csv("../data/batch_resource", header = True))

real_estate_df.write.saveAsTable('real_estate')

24/04/20 19:08:36 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/20 19:08:36 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/20 19:08:37 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/04/20 19:08:37 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore mnestoro@127.0.1.1
24/04/20 19:08:38 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
24/04/20 19:08:40 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
24/04/20 19:08:40 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
24/04/20 19:08:40 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/20 19:08:40 W

In [4]:
real_estate_stream = (
    spark
    .readStream
    .format("table")
    .table("real_estate"))

Streaming DataFrame, like normal DataFrame, is lazily evaluated. What we get is a representation of the stream that we can use to express the series of transformations we want to apply.
Creating a streaming DataFrame does not result in any data actually being consumed or processed until the stream is materialized. 

## Sinks: Output the resulting Data

We materialize a stream by calling `writeStream` on a streaming DataFrame. We need to define *where* and *how* we want the output data to go:
<br>
    **Where**: streaming sink <br>
    **How**: output mode



Calling `writeStream` on a streaming Dataset creates a `DataStreamWriter`. This is a builder instance that provides methods to configure the output behavior of our streaming process.

### Available streaming sinks

1. **Console sink**: A sink prints to the standard output
2. **File sink**: File-based and format-specific sink that writes the results to a filesystem. 
3. **Kafka sink**: A Kafka-specific producer sink that is able to write to one or more Kafka topics.
4. **Memory sink**: Creates an in-memory table using the provided query name as table name. This table receives continuous updates with the results of the stream.
5. **Foreach sink**: Provides a programmatic interface to access the stream contents, one element at the time. Allows to run arbitrary computation on every row record in the output.
6. **Foreach batch sink**: Provides a programmatic sink interface to the complete DataFrame that corresponds to each underlying microbatch of the Structured Streaming execution. Allows to run arbitrary operations and custom logic on the output of each micro-batch. (more on foreach method in later exercises)
7. **Table sink**: Starts the execution of the streaming query, which will continually output results to the given table as new data arrives.


### outputMode

**`append`**(default mode) <br>
Adds only final records to the output stream. A record is considered final when no new records of the incoming stream can modify its value. This is always the case with linear transformations like those resulting from applying projection, filtering, and mapping. This mode guarantees that each row will output only once.

**`update`** <br>
Adds new and updated records since the last trigger to the output stream. Update is meaningful only in the context of an aggregation, where aggregated values change as new records arrive. If more than one incoming record changes a single result, all changes between trigger intervals are collated into one output record.

**`complete`** <br>
Complete mode outputs the complete internal representation of the stream. This mode also relates to aggregations, because for nonaggregated streams, we would need to remember all records seen so far, which is unrealistic. 

### Triggers
`trigger` option lets us specify the frequency at which we want the results to be produced. By default, Structured Streaming will process the input and produce a result as soon as possible. When a trigger is specified, output will be produced at each trigger interval.

* **Fixed interval micro-batches**: Query will be executed with user specified time interval that will dictate the frequency of the query results.
* **Available**: Query will process all the available data and then stop on its own.
* **Continuous**: Switches the execution engine to the experimental continuous engine for low-latency processing.


After this long introduction, let's finaly write output the stream!

`start()` materializes the complete job description into a streaming computation and initiates the internal scheduling process that results in data being consumed from the source, processed, and produced to the sink. `start()` returns a *`StreamingQuery`* instance, which provides a handle that is executing continuosly in the background as new data arrivews. Handle manages the individual life cycle of each query. This means that we can simultaneously start and stop multiple queries independently of one other within the same sparkSession.


In [1]:
query = real_estate_stream.writeStream.format("console").queryName("Table stream").outputMode("append").start()

NameError: name 'real_estate_stream' is not defined

*query* object we have just created is `StreamingQuery` instance. It provides a handle to query that is executing continuously in the background as new data arrives
 

In [6]:

query.isActive

True

In [ ]:
query.lastProgress